In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install finance-datareader

In [3]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from tqdm.notebook import tqdm

In [4]:
path = '/content/drive/MyDrive/stock_2/'
sub1 = pd.read_csv(path + 'prophet/prophet_ensemble_reverse.csv')
sub2 = pd.read_csv(path + 'xgboost_feature.csv')
sub3 = pd.read_csv(path + 'submission_lstm.csv')
sub4 = pd.read_csv(path + 'submission_rnn.csv')
sub5 = pd.read_csv(path + 'submission_arima.csv')

In [5]:
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [6]:
stock_list = stock_list.drop([356])
stock_list.reset_index(drop=True, inplace=True)

In [7]:
start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
public = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {public.shape}', )
display(public.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


,Date
0,2021-11-01
1,2021-11-02
2,2021-11-03
3,2021-11-04
4,2021-11-05


In [8]:
for code in tqdm(sorted(stock_list['종목코드'].values)):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data = data[['Date', 'Close']]
    data.rename(columns={'Close':str(code)}, inplace=True)
    public = pd.merge(public, data, on='Date', how='outer')
public = public.fillna(0)

  0%|          | 0/369 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
start_date = '20211129'
end_date = '20211203'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
private = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {private.shape}', )
display(private.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 48
HOW MANY "Business_days" : (5, 1)


,Date
0,2021-11-29
1,2021-11-30
2,2021-12-01
3,2021-12-02
4,2021-12-03


In [10]:
for code in tqdm(sorted(stock_list['종목코드'].values)):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data = data[['Date', 'Close']]
    data.rename(columns={'Close':str(code)}, inplace=True)
    private = pd.merge(private, data, on='Date', how='outer')
private = private.fillna(0)

  0%|          | 0/369 [00:00<?, ?it/s]

In [11]:
# private.to_csv(path + 'private.csv', index=False)

In [12]:
sub1.drop(columns=['031390'], inplace=True)
sub2.drop(columns=['031390'], inplace=True)
sub3.drop(columns=['031390'], inplace=True)
sub4.drop(columns=['031390'], inplace=True)
sub5.drop(columns=['031390'], inplace=True)

In [13]:
sub1_public = sub1[0:5]
sub2_public = sub2[0:5]
sub3_public = sub3[0:5]
sub4_public = sub4[0:5]
sub5_public = sub5[0:5]

In [14]:
sub1_private = sub1[5:10]
sub2_private = sub2[5:10]
sub3_private = sub3[5:10]
sub4_private = sub4[5:10]
sub5_private = sub5[5:10]

In [15]:
sub1_private.reset_index(drop=True, inplace=True)
sub2_private.reset_index(drop=True, inplace=True)
sub3_private.reset_index(drop=True, inplace=True)
sub4_private.reset_index(drop=True, inplace=True)
sub5_private.reset_index(drop=True, inplace=True)

In [16]:
def nmae(answer_df, submission_df):
    answer = answer_df.iloc[:,1:].to_numpy()
    submission = submission_df.iloc[:,1:].to_numpy()
    target_idx = np.where(answer!=0)
    
    true = answer[target_idx]
    pred = submission[target_idx]
    
    score = np.mean(np.abs(true-pred)/true)
    
    return score

In [17]:
print('np public score : ', nmae(public, sub1_public))
print('xgboost public score : ', nmae(public, sub2_public))
print('lstm public score : ', nmae(public, sub3_public))
print('rnn public score : ', nmae(public, sub4_public))
print('arima public score : ', nmae(public, sub5_public))

np public score :  0.052394742111167014
xgboost public score :  0.052726840131729756
lstm public score :  0.017511769594244806
rnn public score :  0.017336181879796356
arima public score :  0.031215154019193635


In [18]:
print('np private score : ', nmae(private, sub1_private))
print('xgboost private score : ', nmae(private, sub2_private))
print('lstm private score : ', nmae(private, sub3_private))
print('rnn private score : ', nmae(private, sub4_private))
print('airma private score : ', nmae(private, sub5_private))

np private score :  0.06288000975610775
xgboost private score :  0.06781252180479826
lstm private score :  0.10328959905923076
rnn private score :  0.05653804724934986
airma private score :  0.041033534344265245
